In [ ]:
import pygame
import numpy
import time
    
class Grid:
    DEAD = 0
    ALIVE = 1

    def __init__(self, configuration):
        pygame.init()

        self.width = configuration['width']
        self.height = configuration['height']
        self.screen = pygame.display.set_mode((self.height, self.width))

        self.x_cell_number = configuration['x_cell_number']
        self.y_cell_number = configuration['y_cell_number']
        self.x_size = self.width / self.x_cell_number
        self.y_size = self.height / self.y_cell_number
        self.game_state = numpy.zeros((self.x_cell_number, self.y_cell_number))

        self.background = configuration['background'] 
        self.screen.fill(self.background)

    def run(self):
        updatable_game_state = numpy.copy(self.game_state)
        play = True

        while True:
            self.screen.fill(self.background)
            time.sleep(0.1)
            
            events = pygame.event.get()
            
            for event in events:
                if event.type == pygame.KEYDOWN:
                    play = not play
                    
                mouse_click = pygame.mouse.get_pressed()
                
                if sum(mouse_click) > 0:
                    posX, posY = pygame.mouse.get_pos()
                    celX, celY = int(numpy.floor(posX / self.x_cell_number)), int(numpy.floor(posY / self.y_cell_number))
                    updatable_game_state[celX, celY] = not mouse_click(2)

            for y in range(0, self.x_cell_number):
                for x in range(0, self.y_cell_number):
                    if play:
                        self._play(x, y, updatable_game_state)

            self.game_state = updatable_game_state

    def _play(self, x, y, updatable_game_state):
        neighbours = self._neighbours(x, y)

        polygon = [
            ((x)     * self.x_size, (y)     * self.y_size),
            ((x + 1) * self.x_size, (y)     * self.y_size),
            ((x + 1) * self.x_size, (y + 1) * self.y_size),
            ((x)     * self.x_size, (y + 1) * self.y_size)
        ]

        if self.game_state[x, y] == self.DEAD and neighbours == 3:
            updatable_game_state[x, y] = self.ALIVE

        elif self.game_state[x, y] == self.ALIVE and (neighbours < 2 or neighbours > 3):
            updatable_game_state[x, y] = self.DEAD

        if updatable_game_state[x, y] == self.DEAD:
            pygame.draw.polygon(self.screen, (128, 128, 128), polygon, 1)
        else:
            pygame.draw.polygon(self.screen, (255, 255, 255), polygon, 0)
        
    def _neighbours(self, x, y):
        return self.game_state[(x - 1) % self.x_cell_number, (y - 1) % self.y_cell_number] + \
               self.game_state[(x)     % self.x_cell_number, (y - 1) % self.y_cell_number] + \
               self.game_state[(x + 1) % self.x_cell_number, (y - 1) % self.y_cell_number] + \
               self.game_state[(x - 1) % self.x_cell_number, (y)     % self.y_cell_number] + \
               self.game_state[(x + 1) % self.x_cell_number, (y)     % self.y_cell_number] + \
               self.game_state[(x - 1) % self.x_cell_number, (y + 1) % self.y_cell_number] + \
               self.game_state[(x)     % self.x_cell_number, (y + 1) % self.y_cell_number] + \
               self.game_state[(x + 1) % self.x_cell_number, (y + 1) % self.y_cell_number]

class Starters:
    def stick(self, grid):
        grid[5, 3] = grid.ALIVE
        grid[5, 4] = grid.ALIVE
        grid[5, 5] = grid.ALIVE
    
    def automata(self, grid):
        grid[20, 23] = grid.ALIVE
        grid[21, 21] = grid.ALIVE
        grid[21, 23] = grid.ALIVE
        grid[22, 22] = grid.ALIVE
        grid[22, 23] = grid.ALIVE

In [ ]:
configuration = {
    'background': [255, 25, 25],
    'width': 1000,
    'height': 1000,
    'x_cell_number': 50,
    'y_cell_number': 50
}

grid = Grid(configuration)
Starters().stick(grid.game_state)
Starters().automata(grid.game_state)

grid.run()
pygame.quit()